Crime in London
==============



# Introduction

This dataset includes data related to the number of crimes in the different geographic areas of London. The analysis is mostly focused on the period from 2012 to 2016, but the evolution of crime rates from 2009 to 2016 is also provided. The data are divided into major and minor crime categories. The dataset is offered by BigQuery and more information is available [here](https://www.kaggle.com/LondonDataStore/london-crime).


This following analysis attempts to answer questions related to the following:
* Which categories include the highest amount of crimes?
* In which boroughs are the most crimes reported?
* Did the crime rates increase or decrease over time? 




# BigQuery client

In [1]:
from google.cloud import bigquery
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [2]:
# Create a "Client" object
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [3]:
# Reference to the dataset
dataset_ref = client.dataset('london_crime', project='bigquery-public-data')

# Fetch data
dataset = client.get_dataset(dataset_ref)

# List all the tables in the dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:  
    print(table.table_id)

crime_by_lsoa


In [4]:
# Reference to the 'crime_by_lsoa' table
table_ref = dataset_ref.table('crime_by_lsoa')

# Fetch the table
table = client.get_table(table_ref)

# Dataset
client.list_rows(table, max_results=10).to_dataframe()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01032740,City of London,Violence Against the Person,Harassment,0,2016,11
1,E01000005,City of London,Violence Against the Person,Harassment,0,2011,3
2,E01032739,City of London,Violence Against the Person,Harassment,0,2016,12
3,E01000005,City of London,Violence Against the Person,Harassment,0,2011,11
4,E01000005,City of London,Violence Against the Person,Harassment,0,2014,3
5,E01000002,City of London,Violence Against the Person,Harassment,0,2011,5
6,E01000005,City of London,Violence Against the Person,Harassment,0,2011,8
7,E01000005,City of London,Violence Against the Person,Harassment,0,2013,3
8,E01032740,City of London,Violence Against the Person,Harassment,0,2009,11
9,E01032739,City of London,Violence Against the Person,Harassment,0,2008,1


# 1. Total crimes by major category

In [5]:
# Total crimes by major category
query1 = """

    SELECT major_category, SUM(value) AS total_crimes
                FROM `bigquery-public-data.london_crime.crime_by_lsoa`
                WHERE year BETWEEN 2012 AND 2016
                GROUP BY major_category
                HAVING total_crimes > 0
                ORDER BY total_crimes DESC
        """

In [6]:
# Execute query
crimes_by_major_category = client.query(query1).result().to_dataframe()

In [7]:
# Crimes by major category (%) chart

# Create percentage column
sum_of_crimes = crimes_by_major_category['total_crimes'].sum()
crimes_by_major_category['total_crimes%'] = (crimes_by_major_category['total_crimes']/sum_of_crimes*100).round(1)

# Barchart
fig = px.bar(crimes_by_major_category, x ='major_category', y='total_crimes%',
        labels={
                 'major_category': 'Major category',
                 'total_crimes%': 'Crimes (%)'
                },
         title='Total crimes by major category (%), 2012-2016')
fig.show()

# 2. Major crime categories over time

In [8]:
# Crimes per month by major category
query2 = """SELECT major_category, SUM(value) AS crimes_per_month, year, month
                FROM `bigquery-public-data.london_crime.crime_by_lsoa`
                WHERE (year BETWEEN 2009 AND 2016)
                GROUP BY major_category, year, month
                HAVING crimes_per_month > 0
                ORDER BY major_category, year, month
        """

In [9]:
# Execute query
crime_by_month = client.query(query2).result().to_dataframe()

In [10]:
# Construct dates column
crime_by_month['date'] = crime_by_month.year.map(str) + '-' + crime_by_month.month.map(str)
crime_by_month['date'] = pd.to_datetime(crime_by_month.date, format='%Y-%m')
del crime_by_month['year'], crime_by_month['month']

# Pivot
crime_by_month = crime_by_month.pivot_table(values='crimes_per_month',
                                index='date',
                                columns=['major_category'])

In [11]:
# Line plots
fig = px.line(crime_by_month, x=crime_by_month.index, y=list(crime_by_month), title='Major crime categories over time')

fig.show()

'Violence against the Person' rises notably after the end of 2013, whereas the other categories are relatively stable in the same period.

# 3. The average number of crimes per year in different boroughs

In [12]:
# Average number of crimes per year for every borough
query3 = """SELECT borough, cast(ROUND(SUM(value)/5,0) as int64) AS average_crimes
                FROM `bigquery-public-data.london_crime.crime_by_lsoa`
                WHERE year BETWEEN 2012 AND 2016
                GROUP BY borough
                ORDER BY average_crimes DESC
        """

In [13]:
# Execute query
average_crimes = client.query(query3).result().to_dataframe()

In [14]:
#Average number of crimes chart
fig = px.bar(average_crimes, x='borough', y='average_crimes',
        labels={
                 'borough': 'Borough',
                 'average_crimes': 'Average number of crimes per year'
                },
         title='Average number of crimes,  2012 - 2016')
fig.show()

The following section provides an insight into what forms of crime are the most common in the top 5 boroughs.

# 4. Most common minor categories in the boroughs with the highest criminality, 2012-2016

In [15]:
# Top 3 most common minor categories for the top 5 boroughs as reported in section 3,
query4 = """
        WITH common_categories AS (
             SELECT borough, minor_category, SUM(value) AS total_crimes,
                 RANK() OVER (PARTITION BY borough ORDER BY SUM(value) DESC) 
                    AS rank_no
             FROM `bigquery-public-data.london_crime.crime_by_lsoa`
             WHERE year BETWEEN 2012 AND 2016
             GROUP BY borough, minor_category
        )
                
        SELECT borough, minor_category, rank_no, total_crimes
            FROM common_categories
            WHERE rank_no <= 3
              AND borough IN ('Westminster','Lambeth','Southwark','Camden','Newham')
            ORDER BY borough, rank_no
        """

In [16]:
# Execute query
common_categories = client.query(query4).result().to_dataframe()

In [17]:
common_categories

,borough,minor_category,rank_no,total_crimes
0,Camden,Other Theft,1,33188
1,Camden,Other Theft Person,2,16221
2,Camden,Harassment,3,9703
3,Lambeth,Other Theft,1,25999
4,Lambeth,Other Theft Person,2,13069
5,Lambeth,Harassment,3,12699
6,Newham,Other Theft,1,20894
7,Newham,Theft From Motor Vehicle,2,13309
8,Newham,Common Assault,3,11766
9,Southwark,Other Theft,1,25149


Crimes related to theft seem to be the most common.

# 5. 'Other theft' compared to other minor categories

In [18]:
# 'Other theft' per month
query5 = """SELECT minor_category, SUM(value) AS theft_per_month, year, month
                FROM `bigquery-public-data.london_crime.crime_by_lsoa`
                WHERE minor_category = 'Other Theft'
                     AND (year BETWEEN 2009 AND 2016)
                GROUP BY minor_category, year, month
                ORDER BY year, month
        """

In [19]:
# Execute query
theft_per_month = client.query(query5).result().to_dataframe()

In [20]:
# Total crimes per month excluding 'Other theft'
query6 = """SELECT SUM(value) AS other_categories, year, month
                FROM `bigquery-public-data.london_crime.crime_by_lsoa`
                WHERE year BETWEEN 2009 AND 2016
                    AND minor_category != 'Other Theft'
                GROUP BY year, month
                ORDER BY year, month
        """

In [21]:
# Execute query
other_categories = client.query(query6).result().to_dataframe()

In [22]:
# Construct dates column
theft_per_month['date'] = theft_per_month.year.map(str) + '-' + theft_per_month.month.map(str)
theft_per_month['date'] = pd.to_datetime(theft_per_month.date, format='%Y-%m')

In [23]:
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{'secondary_y': True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=theft_per_month['date'], y=theft_per_month['theft_per_month'], name='Other theft'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=theft_per_month['date'], y=other_categories['other_categories'], name='Other categories'),
    secondary_y=True,
)

# Figure title
fig.update_layout(
    title_text="'Other theft' compared to other minor categories"
)

# x-axis title
fig.update_xaxes(title_text='Date')

# y-axes titles
fig.update_yaxes(title_text='Other theft', secondary_y=False)
fig.update_yaxes(title_text='Other categories', secondary_y=True)

fig.show()

'Other theft' peaked during 2012 and declined to the previous levels in the following years. The rest of the minor categories reached a minimum during 2014 but increased again in 2015-2016. As shown in section 2, violence against the person might have been related to this surge.